In [1]:
%uv pip install git+https://github.com/muellerzr/nbdistributed

Using Python 3.12.6 environment at: /usr/local
Updating https://github.com/muellerzr/nbdistributed (HEAD)
Updating https://github.com/muellerzr/nbdistributed (HEAD)
Updating https://github.com/muellerzr/nbdistributed (HEAD)
Updating https://github.com/muellerzr/nbdistributed (HEAD)
Updating https://github.com/muellerzr/nbdistributed (HEAD)
Updating https://github.com/muellerzr/nbdistributed (HEAD)
 Updated https://github.com/muellerzr/nbdistributed (822f9b5)
Resolved 120 packages in 1.11s
⠙ Preparing packages... (0/40)
⠙ Preparing packages... (0/40)
⠙ Preparing packages... (0/40)
prometheus-client ------------------------------     0 B/59.71 KiB
⠙ Preparing packages... (0/40)
prometheus-client ------------------------------ 14.90 KiB/59.71 KiB
⠙ Preparing packages... (0/40)
json5      ------------------------------     0 B/35.27 KiB
prometheus-client ------------------------------ 14.90 KiB/59.71 KiB
⠙ Preparing packages... (0/40)
json5      ------------------------------ 14.95 KiB/35.

In [2]:
import os
os.environ['NBDISTRIBUTED_TIMEOUT'] = '3000'
%load_ext nbdistributed

In [3]:
%dist_init --num-processes 2

Starting 2 distributed workers...
✓ Successfully started 2 workers
Available commands:
  %%distributed - Execute code on all ranks (explicit)
  %%rank [0,n] - Execute code on specific ranks
  %sync - Synchronize all ranks
  %dist_status - Show worker status
  %dist_mode - Toggle automatic distributed mode
  %dist_shutdown - Shutdown workers

🚀 Distributed mode active: All cells will now execute on workers automatically!
   Magic commands (%, %%) will still execute locally as normal.

🐍 Below are auto-imported and special variables auto-generated into the namespace to use
  `torch`
  `dist`: `torch.distributed` import alias
  `rank` (`int`): The local rank
  `world_size` (`int`): The global world size
  `gpu_id` (`int`): The specific GPU ID assigned to this worker
  `device` (`torch.device`): The current PyTorch device object (e.g. `cuda:1`)


<IPython.core.display.Javascript object>

In [6]:
%%rank [0]

tensor = torch.tensor([1,2,3], dtype=torch.float32).to(device); tensor


🔹 Rank 0:
  tensor([1., 2., 3.])


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [7]:
%%rank [1]

tensor = torch.empty(3, dtype=torch.float32).to(device); tensor


🔹 Rank 1:
  tensor([0., 0., 0.])


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [5]:
dist.broadcast(tensor, src=0); tensor


🔹 Rank 0:
  tensor([1., 2., 3.])

🔹 Rank 1:
  tensor([1., 2., 3.])


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
%dist_status

Distributed cluster status (2 processes):
Rank 0: ✓ PID 53924
  ├─ Device: CPU
  └─ Status: Running

Rank 1: ✓ PID 53925
  ├─ Device: CPU
  └─ Status: Running



<IPython.core.display.Javascript object>

In [7]:
%%rank [1]

tensor = torch.empty(3, dtype=torch.float32).to(device)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
def tinfo(t, before=True):
    word = "before" if before else "after"
    print(f"""Rank: {rank} tensor {word} op:
    - Shape: {t.shape}
    - Device: {t.device}
    - Dtype: {t.dtype}
    - Value: {t}
""")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Point -> Point

These send data from A -> B. It can be asynchronous or not

## Synchronous

Needs to happen all in one call/at the same time. Full stop, everything instantly communicates

In [9]:
if rank == 0:
    t = torch.tensor([1,2,3], dtype=torch.float32).to(device)
    tinfo(t)
    dist.send(t, dst=1)
else:
    t = torch.empty(3, dtype=torch.float32).to(device)
    tinfo(t)
    dist.recv(t, src=0)
    tinfo(t, False)


🔹 Rank 0:
  Rank: 0 tensor before op:
    - Shape: torch.Size([3])
    - Device: cpu
    - Dtype: torch.float32
    - Value: tensor([1., 2., 3.])

🔹 Rank 1:
  Rank: 1 tensor before op:
    - Shape: torch.Size([3])
    - Device: cpu
    - Dtype: torch.float32
    - Value: tensor([0., 0., 0.])
  Rank: 1 tensor after op:
    - Shape: torch.Size([3])
    - Device: cpu
    - Dtype: torch.float32
    - Value: tensor([1., 2., 3.])


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

.| explain dist.send

As the name implies *send* will send a tensor *somewhere* from some place to another. This does so in a **blocking** manor, meaning that we can't move on and do any computation on the sending GPU until all GPUs are done receiving the tensor

.| explain dist.recv

*recv* will **receive** the tensor which was sent (also blocking as a result). The receiving tensor can be anything as long as the shape and dtype are appropriate

.| explain torch.empty

We don't *need* to use an empty tensor, however it's handy for our visualization (in all other cases the data would just be overriden with the result)

## Async

Can push tensor and still run other things afterwards, doesn't want for communication.

However, in general you should then wait for it to be completed before going to the next step

In [6]:
%%rank [0]

t = torch.tensor([1,2,3], dtype=torch.float32).to(device)
request = dist.isend(t, dst=1)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

.| explain dist.isend

Rather than doing `send`, asynchronous operations use an `i` notation

.| explain dst=1

Similar to `dist.send`, `isend` passes in the same arguments, such as the destination rank

In [4]:
%%rank [1]

t = torch.empty(3, dtype=torch.float32).to(device)
print(f't before: {t}')
request = dist.irecv(t, src=0)
print(f't after: {t}')


🔹 Rank 1:
  t before: tensor([0., 0., 0.])
  t after: tensor([0., 0., 0.])


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [7]:
t


🔹 Rank 0:
  tensor([1., 2., 3.])

🔹 Rank 1:
  tensor([1., 2., 3.])

❌ Error on Rank 2: name 't' is not defined
Traceback (most recent call last):
  File "/Users/xiaoyuouyang/projects/maven-distributed-training/.venv/lib/python3.12/site-packages/nbdistributed/worker.py", line 284, in _execute_code_streaming
    result = eval(compile(tree, '<string>', 'eval'), self.namespace)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 1, in <module>
NameError: name 't' is not defined



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
%%distributed

request.wait()


🔹 Rank 0:
  True

🔹 Rank 1:
  True

❌ Error on Rank 2: name 'request' is not defined
Traceback (most recent call last):
  File "/Users/xiaoyuouyang/projects/maven-distributed-training/.venv/lib/python3.12/site-packages/nbdistributed/worker.py", line 284, in _execute_code_streaming
    result = eval(compile(tree, '<string>', 'eval'), self.namespace)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 1, in <module>
NameError: name 'request' is not defined



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Doing async still is better because:

- non-blocking/overlapping computation within the communication

## One -> All

In [8]:
%%rank [0]

tensor = torch.tensor([1,2,3], dtype=torch.float32).to(device); tensor


🔹 Rank 0:
  tensor([1., 2., 3.])


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
%%rank [1]

tensor = torch.empty(3, dtype=torch.float32).to(device); tensor


=== Ranks [1] ===

--- Rank 1 ---
tensor([0., 0., 0.], device='cuda:4')


In [9]:
dist.broadcast(tensor, src=0); tensor


🔹 Rank 0:
  tensor([1., 2., 3.])

🔹 Rank 1:
  tensor([1., 2., 3.])


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Scatter will take a tensor and chunkify it between other GPUs

In [16]:
%%rank [0]
scatter_list = [torch.tensor([i, i+1]).to(device) for i in range(0,4,2)]; print(scatter_list)


🔹 Rank 0:
  [tensor([0, 1]), tensor([2, 3])]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [17]:
%%rank [1]
scatter_list = None

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [18]:
%%distributed
output = torch.empty(2, dtype=torch.int64).to(device)
dist.scatter(output, scatter_list, src=0)
print(f'Rank {rank} received: {output}')


🔹 Rank 0:
  Rank 0 received: tensor([0, 1])

🔹 Rank 1:
  Rank 1 received: tensor([2, 3])


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Reduction

## All -> All

In [15]:
%%distributed

t = torch.tensor([0+rank, 1+rank, 2+rank]).to(device)
print(f't before op: {t}')
dist.all_reduce(t, op=dist.ReduceOp.SUM)
print(f't after op: {t}')


=== All ranks ===

--- Rank 0 ---
t before op: tensor([0, 1, 2], device='cuda:3')
t after op: tensor([1, 3, 5], device='cuda:3')


--- Rank 1 ---
t before op: tensor([1, 2, 3], device='cuda:4')
t after op: tensor([1, 3, 5], device='cuda:4')



In [16]:
%%distributed

t = torch.tensor([0+rank, 1+rank, 2+rank]).to(device)
print(f't before op: {t}')
dist.all_reduce(t, op=dist.ReduceOp.MAX)
print(f't after op: {t}')


=== All ranks ===

--- Rank 0 ---
t before op: tensor([0, 1, 2], device='cuda:3')
t after op: tensor([1, 2, 3], device='cuda:3')


--- Rank 1 ---
t before op: tensor([1, 2, 3], device='cuda:4')
t after op: tensor([1, 2, 3], device='cuda:4')



In [17]:
%%distributed

t = torch.tensor([0+rank, 1+rank, 2+rank]).to(device)
print(f't before op: {t}')
dist.all_reduce(t, op=dist.ReduceOp.MIN)
print(f't after op: {t}')


=== All ranks ===

--- Rank 0 ---
t before op: tensor([0, 1, 2], device='cuda:3')
t after op: tensor([0, 1, 2], device='cuda:3')


--- Rank 1 ---
t before op: tensor([1, 2, 3], device='cuda:4')
t after op: tensor([0, 1, 2], device='cuda:4')



In [18]:
%%distributed

t = torch.tensor([0+rank, 1+rank, 2+rank]).to(device)
print(f't before op: {t}')
dist.all_reduce(t, op=dist.ReduceOp.PRODUCT)
print(f't after op: {t}')


=== All ranks ===

--- Rank 0 ---
t before op: tensor([0, 1, 2], device='cuda:3')
t after op: tensor([0, 2, 6], device='cuda:3')


--- Rank 1 ---
t before op: tensor([1, 2, 3], device='cuda:4')
t after op: tensor([0, 2, 6], device='cuda:4')



### All -> One

Reduces all tensors from all ranks to `dst`, useful for things like calculating metrics and printing only on rank0

In [4]:
%%distributed

t = torch.tensor([0+rank, 1+rank, 2+rank]).to(device)
print(f't before op: {t}, rank {rank}')
dist.reduce(t, dst=0, op=dist.ReduceOp.SUM)
print(f't after op: {t}, rank {rank}')


🔹 Rank 0:
  t before op: tensor([0, 1, 2], device='cuda:0'), rank 0

🔹 Rank 1:
  t before op: tensor([1, 2, 3], device='cuda:1'), rank 1
  t after op: tensor([1, 3, 5], device='cuda:0'), rank 0
  t after op: tensor([1, 2, 3], device='cuda:1'), rank 1


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Gathering

Assume each rank has different data, gather them all into various lists

In [7]:
tensor_list = [torch.zeros(3, dtype=torch.int64).to(device) for _ in range(2)]
t = torch.tensor([0+rank, 1+rank, 2+rank], dtype=torch.int64).to(device)
print(f'Before op: {tensor_list}, rank {rank}')
dist.all_gather(tensor_list, t)
print(f'After op: {tensor_list}, rank {rank}')


🔹 Rank 0:
  Before op: [tensor([0, 0, 0], device='cuda:0'), tensor([0, 0, 0], device='cuda:0')], rank 0
  After op: [tensor([0, 1, 2], device='cuda:0'), tensor([1, 2, 3], device='cuda:0')], rank 0

🔹 Rank 1:
  Before op: [tensor([0, 0, 0], device='cuda:1'), tensor([0, 0, 0], device='cuda:1')], rank 1
  After op: [tensor([0, 1, 2], device='cuda:1'), tensor([1, 2, 3], device='cuda:1')], rank 1


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [10]:
tensor_list = torch.zeros(2,3, dtype=torch.int64).to(device) 
t = torch.tensor([0+rank, 1+rank, 2+rank], dtype=torch.int64).to(device)
print(f'Before op: {tensor_list}, rank {rank}')
dist.all_gather_into_tensor(tensor_list, t)
print(f'After op: {tensor_list}, rank {rank}')


🔹 Rank 0:
  Before op: tensor([[0, 0, 0],
        [0, 0, 0]], device='cuda:0'), rank 0
  After op: tensor([[0, 1, 2],
        [1, 2, 3]], device='cuda:0'), rank 0

🔹 Rank 1:
  Before op: tensor([[0, 0, 0],
        [0, 0, 0]], device='cuda:1'), rank 1
  After op: tensor([[0, 1, 2],
        [1, 2, 3]], device='cuda:1'), rank 1


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [14]:
%dist_shutdown

Shutting down distributed workers (nuclear option)...
Starting force shutdown...
Force shutdown completed
Distributed workers shutdown
📱 Normal cell execution restored


<IPython.core.display.Javascript object>